# **Project Supervised learning**
## **Ricardo Villarreal Bazán - A01666859**

### **Introduction**

Supervised learning is a core paradigm in machine learning whereby an algorithm learns to map input features to known output labels by training on labeled examples. In classification tasks, the objective is to predict a discrete category for each instance—such as determining the obesity level of an individual based on eating and lifestyle habits. Decision tree classifiers are particularly well suited for this purpose because they can handle mixed data types (numeric and categorical), capture non‐linear relationships, and produce interpretable, rule‐based models.

A decision tree builds a hierarchical set of binary (or multiway) splits on input features, partitioning the dataset into progressively purer subsets until a stopping criterion is met. During training, the tree selects the feature and threshold that maximally reduce class impurity (e.g., using Gini index or information gain). Once constructed, each path from root to leaf can be read as an “if–then” rule—making it straightforward to explain why a particular individual’s eating habits and physical measurements lead to a predicted obesity category.


In [16]:
from ucimlrepo import fetch_ucirepo 

from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# fetch dataset 
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 
  
# data (as pandas dataframes) 
X = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.features 
y = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.targets 
  
# metadata 
print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.metadata) 
  
# variable information 
print(estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.variables) 


{'uci_id': 544, 'name': 'Estimation of Obesity Levels Based On Eating Habits and Physical Condition ', 'repository_url': 'https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition', 'data_url': 'https://archive.ics.uci.edu/static/public/544/data.csv', 'abstract': 'This dataset include data for the estimation of obesity levels in individuals from the countries of Mexico, Peru and Colombia, based on their eating habits and physical condition. ', 'area': 'Health and Medicine', 'tasks': ['Classification', 'Regression', 'Clustering'], 'characteristics': ['Multivariate'], 'num_instances': 2111, 'num_features': 16, 'feature_types': ['Integer'], 'demographics': ['Gender', 'Age'], 'target_col': ['NObeyesdad'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2019, 'last_updated': 'Tue Sep 10 2024', 'dataset_doi': '10.24432/C5H31Z', 'creators': [], 'intro_paper': {'ID': 358, 'type': 

In [17]:
X

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation


In [18]:
y

,NObeyesdad
0,Normal_Weight
1,Normal_Weight
2,Normal_Weight
3,Overweight_Level_I
4,Overweight_Level_II
...,...
2106,Obesity_Type_III
2107,Obesity_Type_III
2108,Obesity_Type_III
2109,Obesity_Type_III


### 1.1 Context and Objective

The “Estimation of Obesity Levels Based on Eating Habits and Physical Condition” dataset (UCI ML Repository, ID = 544) was collected with the goal of predicting an individual’s obesity category (e.g., Normal_Weight, Overweight_Level_I, Obesity_Type_I, etc.) from a combination of self‐reported eating habits and physical measurements. In practice, this kind of dataset can be used to build classification models that alert health professionals or end users about potential obesity risks.

### 1.2 Data Source

- **Repository**: UCI Machine Learning Repository  
- **Fetch command**:
  ```python
  pip install ucimlrepo
  from ucimlrepo import fetch_ucirepo

  dataset = fetch_ucirepo(id=544)
  X = dataset.data.features
  y = dataset.data.targets



---

## 2. High-Level Dataset Structure


### 2.1 Features (X)

There are 17 input attributes (columns) in `X`. They can be grouped roughly as follows:

1. **Demographic / Anthropometric**
   - `Gender` (categorical: “Male” / “Female”)
   - `Age` (numeric: integer, in years)
   - `Height` (numeric: in meters or centimeters—check units)
   - `Weight` (numeric: in kilograms)

2. **Family History**
   - `family_history_with_overweight` (binary: “yes” / “no”)

3. **Eating Habits (self-reported)**
   - `FAVC` (Frequent consumption of high caloric food; categorical: “yes” / “no”)
   - `FCVC` (Frequency of vegetable consumption; integer scale 0–3, where higher means more frequent)
   - `NCP` (Number of main meals per day; integer)
   - `CAEC` (Consumption of food between meals; categorical: “no” / “Sometimes” / “Frequently” / “Always”)
   - `SMOKE` (Binary: “yes” / “no”)
   - `CH2O` (Daily water consumption in liters; numeric)
   - `SCC` (Calories consumption monitoring; categorical: “yes” / “no”)
   - `FAF` or `FAHF` (Frequency of high caloric food consumption; integer 0–3)
   - `TUE` (Frequency of physical activity; integer 0–3)
   - `CALC` (Time using electronic devices; categorical: “no” / “Sometimes” / “Frequently” / “Always”)
   - `MTRANS` (Mode of transportation; categorical: “Automobile” / “Motorbike” / “Bike” / “Public_Transportation” / “Walking”)

4. **Physical Condition / Activity**
   - `Smoke` (often redundant with SMOKE—verify naming)
   - `FCVC` (frequency of vegetable consumption—might double as health habit)
   - `TUE` (frequency of training)
   - Note: some attributes appear in multiple groupings (e.g., `SMOKE` also appears under “lifestyle”). Always cross-check the variable descriptions below.

### 2.2 Target (y)

The column `y` (or `dataset.data.targets`) contains one of **seven** categories:

1. `Insufficient_Weight`  
2. `Normal_Weight`  
3. `Overweight_Level_I`  
4. `Overweight_Level_II`  
5. `Obesity_Type_I`  
6. `Obesity_Type_II`  
7. `Obesity_Type_III`  

Each record in `X` corresponds to exactly one of these obesity categories in `y`.


In [19]:
print("Shape of X:", X.shape)
print("Target distribution:\n", y.value_counts())
print("\nData types & nulls:\n", X.info())
print(X.describe(include='all'))


Shape of X: (2111, 16)
Target distribution:
 NObeyesdad         
Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8 

----

## 3. Classifier  


### **Random Forest Classifier**

A Random Forest builds an ensemble of decision trees on bootstrapped (randomly sampled) subsets of the training data, and at each split it considers a random subset of features. Each tree votes on the class, and the forest predicts by majority vote (or averaged probabilities).

**Key Advantages:**

Robustness & Accuracy: By averaging many trees, it reduces overfitting compared to a single decision tree and often achieves high predictive performance out of the box.
Handles Mixed Data Types: Works “out of the box” with numeric and categorical features without needing extensive preprocessing (beyond one-hot encoding if necessary).
Implicit Feature Importance: Provides a straightforward ranking of feature importances, helping to identify which inputs matter most.
Resilient to Outliers & Noise: Since each tree sees a different subset of data and features, individual noisy points have limited influence on the overall model.

**Key Disadvantages:**

Less Interpretable: Unlike a single decision tree (where you can read “if-then” rules), a Random Forest’s aggregate of hundreds of trees is hard to visualize or explain in simple terms.
Slower Prediction & Training: Training many trees (and tuning hyperparameters like number of trees, depth, etc.) can be computationally intensive, especially on large datasets.
Memory Consumption: Storing dozens or hundreds of trees requires more RAM than a single model.
Potential Overfitting with Noisy Features: Though more robust than one tree, if there are many irrelevant or highly noisy features, the forest can still learn spurious patterns unless properly tuned (e.g., via limiting tree depth or feature subsampling).


----


## 4. Preprocessing

In [20]:
# 2. Preprocessing
# Map binary categorical columns ('yes'/'no') to 0/1
binary_cols = ['family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC']
for col in binary_cols:
    X[col] = X[col].map({'no': 0, 'yes': 1})

# One-hot encode categorical columns with >2 categories
categorical_cols = ['CAEC', 'CALC', 'MTRANS', 'Gender']
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# 3. Train–test split (stratify on target to preserve class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Baseline Random Forest Classifier (default params)
rf_baseline = RandomForestClassifier(random_state=42)
rf_baseline.fit(X_train, y_train)

# Evaluate baseline
y_pred_baseline = rf_baseline.predict(X_test)
print("Baseline Random Forest Classification Report:\n")
print(classification_report(y_test, y_pred_baseline))
print("Baseline Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_baseline))

# 5. Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# Stratified 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=cv,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

# Best parameters from GridSearchCV
best_params = grid_search.best_params_
print("\nBest Hyperparameters:\n", best_params)

# 6. Evaluate the best estimator on the test set
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

print("\nTuned Random Forest Classification Report:\n")
print(classification_report(y_test, y_pred_best))
print("Tuned Random Forest Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred_best))

# 7. Feature importance (top 10)
importances = best_rf.feature_importances_
feature_names = X.columns
feature_importances = pd.Series(importances, index=feature_names).sort_values(ascending=False)

print("\nTop 10 Feature Importances:\n")
print(feature_importances.head(10))

/var/folders/nv/cbzb774d1579x2pzrfcgl4tm0000gn/T/ipykernel_16407/781244788.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].map({'no': 0, 'yes': 1})
/var/folders/nv/cbzb774d1579x2pzrfcgl4tm0000gn/T/ipykernel_16407/781244788.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].map({'no': 0, 'yes': 1})
/var/folders/nv/cbzb774d1579x2pzrfcgl4tm0000gn/T/ipykernel_16407/781244788.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

Baseline Random Forest Classification Report:

                     precision    recall  f1-score   support

Insufficient_Weight       1.00      0.94      0.97        54
      Normal_Weight       0.75      0.95      0.84        58
     Obesity_Type_I       0.96      0.96      0.96        70
    Obesity_Type_II       1.00      0.98      0.99        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0.94      0.84      0.89        58
Overweight_Level_II       0.98      0.91      0.95        58

           accuracy                           0.94       423
          macro avg       0.95      0.94      0.94       423
       weighted avg       0.95      0.94      0.94       423

Baseline Confusion Matrix:

[[51  3  0  0  0  0  0]
 [ 0 55  0  0  0  3  0]
 [ 0  2 67  0  0  0  1]
 [ 0  0  1 59  0  0  0]
 [ 0  1  0  0 64  0  0]
 [ 0  8  1  0  0 49  0]
 [ 0  4  1  0  0  0 53]]
Fitting 5 folds for each of 72 candidates, totalling 360 fits


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packag

[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   3.2s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   3.1s


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


...

[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   5.2s


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   4.7s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   4.7s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   3.1s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   2.3s


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Best Hyperparameters:
 {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'n_estimators': 300}

Tuned Random Forest Classification Report:

                     precision    recall  f1-score   support

Insufficient_Weight       1.00      0.93      0.96        54
      Normal_Weight       0.77      0.97      0.85        58
     Obesity_Type_I       0.97      0.96      0.96        70
    Obesity_Type_II       1.00      0.98      0.99        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0.96      0.86      0.91        58
Overweight_Level_II       0.98      0.95      0.96        58

           accuracy                           0.95       423
          macro avg       0.95      0.95      0.95       423
       weighted avg       0.96      0.95      0.95       423

Tuned Random Forest Confusion Matrix:

[[50  4  0  0  0  0  0]
 [ 0 56  0  0  0  2  0]
 [ 0  2 67  0  0  0  1]
 [ 0  0  1 59  0  0  0]
 [ 0  1  0  0 64  0  0]
 [ 0  8  0

In [21]:
from sklearn.model_selection import train_test_split

# After preprocessing X and y:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("X_train shape:", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train distribution:\n", y_train.value_counts())
print("y_test distribution:\n", y_test.value_counts())


X_train shape: (1688, 23)
X_test shape:  (423, 23)
y_train distribution:
 NObeyesdad         
Obesity_Type_I         281
Obesity_Type_III       259
Obesity_Type_II        237
Overweight_Level_I     232
Overweight_Level_II    232
Normal_Weight          229
Insufficient_Weight    218
Name: count, dtype: int64
y_test distribution:
 NObeyesdad         
Obesity_Type_I         70
Obesity_Type_III       65
Obesity_Type_II        60
Normal_Weight          58
Overweight_Level_I     58
Overweight_Level_II    58
Insufficient_Weight    54
Name: count, dtype: int64


In [22]:

# 1. Define the scoring dictionary
scoring = {
    'accuracy'      : 'accuracy',
    'f1_weighted'   : 'f1_weighted',
    'f1_macro'      : 'f1_macro'
}

# 2. Reuse your parameter grid
param_grid = {
    'n_estimators'     : [100, 200, 300],
    'max_depth'        : [None, 10, 20, 30],
    'min_samples_leaf' : [1, 2, 4],
    'criterion'        : ['gini', 'entropy']
}

# 3. Set up stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. GridSearchCV with multiple metrics, refitting on weighted F1
rf = RandomForestClassifier(random_state=42)
grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=scoring,
    refit='f1_weighted',    # choose the best model by weighted F1
    cv=cv,
    n_jobs=-1,
    verbose=2
)

# 5. Run the search
grid.fit(X_train, y_train)

# 6. Report results
print("Best params (by weighted F1):", grid.best_params_)
print("\nCV results summary:")
for metric in scoring:
    mean_score = grid.cv_results_[f'mean_test_{metric}'][grid.best_index_]
    print(f"  {metric:<12}: {mean_score:.3f}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packag

[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   1.9s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   1.9s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   1.9s
[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   1.9s


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packag

[CV] END criterion=gini, max_depth=None, min_samples_leaf=1, n_estimators=100; total time=   1.2s


...

[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   3.0s


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   2.8s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   2.7s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   2.3s
[CV] END criterion=entropy, max_depth=30, min_samples_leaf=4, n_estimators=300; total time=   1.7s


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best params (by weighted F1): {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'n_estimators': 300}

CV results summary:
  accuracy    : 0.947
  f1_weighted : 0.947
  f1_macro    : 0.945


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 1. Instantiate base classifiers (you can replace params with your tuned values)
dt_clf = DecisionTreeClassifier(random_state=42, max_depth=20, min_samples_leaf=2)
rf_clf = RandomForestClassifier(**best_params, random_state=42)  
lr_clf = LogisticRegression(max_iter=1000, random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=5)
gnb_clf = GaussianNB()

# 2. Voting Ensemble (soft voting)
voting_clf = VotingClassifier(
    estimators=[
        ('dt', dt_clf),
        ('rf', rf_clf),
        ('lr', lr_clf),
        ('knn', knn_clf),
        ('gnb', gnb_clf)
    ],
    voting='soft',
    n_jobs=-1
)

voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)

print("Voting Classifier Report:\n")
print(classification_report(y_test, y_pred_voting))
print("Voting Confusion Matrix:\n", confusion_matrix(y_test, y_pred_voting))

# 3. Stacking Ensemble
stacking_clf = StackingClassifier(
    estimators=[
        ('dt', dt_clf),
        ('rf', rf_clf),
        ('lr', lr_clf),
        ('knn', knn_clf)
    ],
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5,
    n_jobs=-1
)

stacking_clf.fit(X_train, y_train)
y_pred_stack = stacking_clf.predict(X_test)

print("\nStacking Classifier Report:\n")
print(classification_report(y_test, y_pred_stack))
print("Stacking Confusion Matrix:\n", confusion_matrix(y_test, y_pred_stack))

/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.

Voting Classifier Report:

                     precision    recall  f1-score   support

Insufficient_Weight       0.94      0.91      0.92        54
      Normal_Weight       0.81      0.79      0.80        58
     Obesity_Type_I       0.97      0.97      0.97        70
    Obesity_Type_II       0.94      0.98      0.96        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0.84      0.88      0.86        58
Overweight_Level_II       0.98      0.95      0.96        58

           accuracy                           0.93       423
          macro avg       0.93      0.92      0.92       423
       weighted avg       0.93      0.93      0.93       423

Voting Confusion Matrix:
 [[49  5  0  0  0  0  0]
 [ 3 46  0  0  0  9  0]
 [ 0  0 68  1  0  1  0]
 [ 0  0  1 59  0  0  0]
 [ 0  0  0  1 64  0  0]
 [ 0  6  0  0  0 51  1]
 [ 0  0  1  2  0  0 55]]


/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/ricardovillarreal/anaconda3/envs/DATAANALISIS/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.


Stacking Classifier Report:

                     precision    recall  f1-score   support

Insufficient_Weight       0.98      0.91      0.94        54
      Normal_Weight       0.83      0.91      0.87        58
     Obesity_Type_I       0.97      1.00      0.99        70
    Obesity_Type_II       0.98      0.98      0.98        60
   Obesity_Type_III       1.00      0.98      0.99        65
 Overweight_Level_I       0.89      0.88      0.89        58
Overweight_Level_II       0.98      0.95      0.96        58

           accuracy                           0.95       423
          macro avg       0.95      0.95      0.95       423
       weighted avg       0.95      0.95      0.95       423

Stacking Confusion Matrix:
 [[49  5  0  0  0  0  0]
 [ 1 53  0  0  0  4  0]
 [ 0  0 70  0  0  0  0]
 [ 0  0  1 59  0  0  0]
 [ 0  0  0  1 64  0  0]
 [ 0  6  0  0  0 51  1]
 [ 0  0  1  0  0  2 55]]


## 5. Performance Metrics & Test Set Results

### 5.1 Selected Performance Metrics
- **Accuracy**  
  Overall fraction of correct predictions—an easy‐to‐interpret overall score, but can mask poor minority‐class performance.
- **Macro F1-Score**  
  Unweighted average of per-class F1 scores; treats each class equally, so that mistakes on small classes aren’t hidden by large ones.
- **Weighted F1-Score**  
  Average of per-class F1 scores weighted by class support; balances precision and recall while accounting for the class distribution.

### 5.2 Test Set Results

| Classifier            | Accuracy | Macro F1 | Weighted F1 |
|-----------------------|---------:|---------:|------------:|
| Baseline Random Forest| 0.94     | 0.94     | 0.94        |
| Tuned Random Forest   | 0.95     | 0.95     | 0.96        |
| Voting Ensemble       | 0.93     | 0.92     | 0.93        |
| Stacking Ensemble     | 0.95     | 0.95     | 0.95        |

- **Baseline RF** (accuracy 0.94, macro F1 0.94, weighted F1 0.94) : contentReference[oaicite:0]{index=0}  
- **Tuned RF** (accuracy 0.95, macro F1 0.95, weighted F1 0.96) : contentReference[oaicite:1]{index=1}  
- **Voting Ensemble** (accuracy 0.93, macro F1 0.92, weighted F1 0.93) : contentReference[oaicite:2]{index=2}  
- **Stacking Ensemble** (accuracy 0.95, macro F1 0.95, weighted F1 0.95) : contentReference[oaicite:3]{index=3}  
